In [34]:
import pandas as pd
import numpy as np
import pandas.util.testing as tm
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

__IMPORT I BADANIE DANYCH__

In [35]:
df = pd.read_csv('data/AmesHousing.csv')

__BRAKI W DANYCH - UZUPŁNIENIE DANYCH__

In [37]:
num_missing = df.isna().sum()
num_missing = num_missing[num_missing > 0]
percent_missing = num_missing * 100 / df.shape[0]
pd.concat([num_missing, percent_missing], axis=1, 
          keys=['Missing Values', 'Percentage']).\
          sort_values(by="Missing Values", ascending=False)

,Missing Values,Percentage
Pool QC,2917,99.56
Misc Feature,2824,96.38
Alley,2732,93.24
Fence,2358,80.48
Fireplace Qu,1422,48.53
Lot Frontage,490,16.72
Garage Cond,159,5.43
Garage Qual,159,5.43
Garage Finish,159,5.43
Garage Yr Blt,159,5.43


In [38]:
missing_col = df.columns[df.isnull().any()]
missing_col

Index(['Lot Frontage', 'Alley', 'Mas Vnr Type', 'Mas Vnr Area', 'Bsmt Qual',
       'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin SF 1',
       'BsmtFin Type 2', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF',
       'Electrical', 'Bsmt Full Bath', 'Bsmt Half Bath', 'Fireplace Qu',
       'Garage Type', 'Garage Yr Blt', 'Garage Finish', 'Garage Cars',
       'Garage Area', 'Garage Qual', 'Garage Cond', 'Pool QC', 'Fence',
       'Misc Feature'],
      dtype='object')

In [39]:
df["Pool QC"].fillna("No Pool", inplace=True)
df['Misc Feature'].fillna('No feature', inplace=True)
df['Alley'].fillna('No Alley', inplace=True)
df['Fence'].fillna('No Fence', inplace=True)
df['Fireplace Qu'].fillna('No Fireplace', inplace=True)
df['Lot Frontage'].fillna(0, inplace=True)
df['Mas Vnr Area'].fillna(0, inplace=True)
df['Mas Vnr Type'].fillna("None", inplace=True)
df['Electrical'].fillna(df['Electrical'].mode()[0], inplace=True)

In [40]:
garage_columns = [col for col in df.columns if col.startswith("Garage")]
df[df['Garage Cars'].isna()][garage_columns]

,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond
2236,Detchd,nan,NaN,nan,nan,NaN,NaN


In [41]:
df[~pd.isna(df['Garage Type']) & 
        pd.isna(df['Garage Qual'])][garage_columns]

,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond
1356,Detchd,nan,NaN,1.00,360.00,NaN,NaN
2236,Detchd,nan,NaN,nan,nan,NaN,NaN


In [42]:
df['Garage Cars'].fillna(0, inplace=True)
df['Garage Area'].fillna(0, inplace=True)

df.loc[~pd.isna(df['Garage Type']) & 
            pd.isna(df['Garage Qual']), "Garage Type"] = "No Garage"

for col in ['Garage Type', 'Garage Finish', 'Garage Qual', 'Garage Cond']:
    df[col].fillna('No Garage', inplace=True)
    
df['Garage Yr Blt'].fillna(0, inplace=True)

In [43]:
bsmt_columns = [col for col in df.columns if "Bsmt" in col]
df[df['Bsmt Half Bath'].isna()][bsmt_columns]

,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Bsmt Full Bath,Bsmt Half Bath
1341,NaN,NaN,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan
1497,NaN,NaN,NaN,NaN,0.00,NaN,0.00,0.00,0.00,nan,nan


In [44]:
df[~pd.isna(df['Bsmt Cond']) & 
        pd.isna(df['Bsmt Exposure'])][bsmt_columns]

,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Bsmt Full Bath,Bsmt Half Bath
66,Gd,TA,NaN,Unf,0.00,Unf,0.00,1595.00,1595.00,0.00,0.00
1796,Gd,TA,NaN,Unf,0.00,Unf,0.00,725.00,725.00,0.00,0.00
2779,Gd,TA,NaN,Unf,0.00,Unf,0.00,936.00,936.00,0.00,0.00


In [45]:
df[~pd.isna(df['Bsmt Cond']) & 
        pd.isna(df['BsmtFin Type 2'])][bsmt_columns]

,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Bsmt Full Bath,Bsmt Half Bath
444,Gd,TA,No,GLQ,1124.00,NaN,479.00,1603.00,3206.00,1.00,0.00


In [46]:
for col in ["Bsmt Half Bath", "Bsmt Full Bath", "Total Bsmt SF", 
            "Bsmt Unf SF", "BsmtFin SF 2", "BsmtFin SF 1"]:
    df[col].fillna(0, inplace=True)

df.loc[~pd.isna(df['Bsmt Cond']) & 
            pd.isna(df['Bsmt Exposure']), "Bsmt Exposure"] = "No"
df.loc[~pd.isna(df['Bsmt Cond']) & 
            pd.isna(df['BsmtFin Type 2']), "BsmtFin Type 2"] = "Unf"

for col in ["Bsmt Exposure", "BsmtFin Type 2", 
            "BsmtFin Type 1", "Bsmt Qual", "Bsmt Cond"]:
    df[col].fillna("No Basement", inplace=True)

In [47]:
df.drop(['PID'], axis=1, inplace=True)

In [48]:
df.isna().values.sum()

0

__Zamiana zmiennych kategorczynych__

In [49]:
mp = {'Ex':4,'Gd':3,'TA':2,'Fa':1,'Po':0}
df['Exter Qual'] = df['Exter Qual'].map(mp)
df['Exter Cond'] = df['Exter Cond'].map(mp)
df['Heating QC'] = df['Heating QC'].map(mp)
df['Kitchen Qual'] = df['Kitchen Qual'].map(mp)

mp = {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'No Basement':0}
df['Bsmt Qual'] = df['Bsmt Qual'].map(mp)
df['Bsmt Cond'] = df['Bsmt Cond'].map(mp)
df['Bsmt Exposure'] = df['Bsmt Exposure'].map(
    {'Gd':4,'Av':3,'Mn':2,'No':1,'No Basement':0})

mp = {'GLQ':6,'ALQ':5,'BLQ':4,'Rec':3,'LwQ':2,'Unf':1,'No Basement':0}
df['BsmtFin Type 1'] = df['BsmtFin Type 1'].map(mp)
df['BsmtFin Type 2'] = df['BsmtFin Type 2'].map(mp)

df['Central Air'] = df['Central Air'].map({'Y':1,'N':0})
df['Functional'] = df['Functional'].map(
    {'Typ':7,'Min1':6,'Min2':5,'Mod':4,'Maj1':3,
     'Maj2':2,'Sev':1,'Sal':0})
df['Fireplace Qu'] = df['Fireplace Qu'].map(
    {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'No Fireplace':0})
df['Garage Finish'] = df['Garage Finish'].map(
    {'Fin':3,'RFn':2,'Unf':1,'No Garage':0})
df['Garage Qual'] = df['Garage Qual'].map(
    {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'No Garage':0})
df['Garage Cond'] = df['Garage Cond'].map(
    {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'No Garage':0})
df['Pool QC'] = df['Pool QC'].map(
    {'Ex':4,'Gd':3,'TA':2,'Fa':1,'No Pool':0})
df['Land Slope'] = df['Land Slope'].map(
    {'Sev': 2, 'Mod': 1, 'Gtl': 0})
df['Fence'] = df['Fence'].map(
    {'GdPrv':4,'MnPrv':3,'GdWo':2,'MnWw':1,'No Fence':0})

In [50]:
df = pd.get_dummies(df)

__standaryzacja danych__

In [51]:
scaler = StandardScaler()
df.loc[:, df.columns != 'SalePrice'] = scaler.fit_transform(
    df.loc[:, df.columns != 'SalePrice'])

__Podział na x i y i zbiory treningowy i testowy__

In [52]:
y = df['SalePrice']
X = df.drop(['SalePrice'], axis=1)

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, random_state=3)

Linear Regression

In [102]:
import re
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LinearRegression,  LogisticRegression
from sklearn.metrics import r2_score
from statsmodels.formula.api import ols

In [60]:


variables = []

for v in list(df.columns):
    x = re.sub(r'[\s\W]', '', v)
    
    if x[0].isdigit():
        x = 'x' + x
    else:
        x
        
    variables.append(x)


df.columns=variables

In [61]:
variables.remove('SalePrice')

In [62]:
X_train.columns = variables
X_test.columns = variables

In [63]:
model_def =  'SalePrice ~' +  ' ' + ' + '.join(variables)

In [64]:
mul_lin_regress = ols(model_def, df).fit()
mul_lin_regress.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.923
Model:                            OLS   Adj. R-squared:                  0.917
Method:                 Least Squares   F-statistic:                     151.5
Date:                Sat, 27 Jun 2020   Prob (F-statistic):               0.00
Time:                        15:19:57   Log-Likelihood:                -33481.
No. Observations:                2930   AIC:                         6.739e+04
Df Residuals:                    2715   BIC:                         6.868e+04
Df Model:                         214                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept              1.808e+05    426.067    424.337      0.000     1.8e+05    1.82e+05
Order                 -4059.0203   5053.194     -0.803      0.422    -1.4e+04    5849.474
MSSubClass            -5185.9975   2153.986     -2.408      0.016   -9409.616    -962.379
LotFrontage             905.4324    535.430      1.691      0.091    -144.460    1955.325
LotArea                3609.9877    600.127      6.015      0.000    2433.236    4786.739
LandSlope                57.6097    608.899      0.095      0.925   -1136.342    1251.561
OverallQual            1.125e+04    947.430     11.874      0.000    9391.610    1.31e+04
OverallCond            6196.0090    643.409      9.630      0.000    4934.387    7457.631
YearBuilt              7973.5957   1544.205      5.164      0.000    4945.660     1.1e+04
YearRemodAdd            504.4908    773.953      0.652      0.515   -1013.105    2022.086
MasVnrArea             6012.0231    717.725      8.376      0.000    4604.680    7419.367
ExterQual              3950.0265    831.720      4.749      0.000    2319.159    5580.894
ExterCond                -7.1385    504.236     -0.014      0.989    -995.864     981.587
BsmtQual               4086.6479   1013.448      4.032      0.000    2099.441    6073.855
BsmtCond              -3153.5125    772.631     -4.082      0.000   -4668.518   -1638.507
BsmtExposure           5262.5852    636.191      8.272      0.000    4015.117    6510.053
BsmtFinType1           -843.2818    747.169     -1.129      0.259   -2308.360     621.796
BsmtFinSF1             8942.2129    635.124     14.079      0.000    7696.838    1.02e+04
BsmtFinType2            -68.2549    779.492     -0.088      0.930   -1596.712    1460.203
BsmtFinSF2             1312.3434    744.473      1.763      0.078    -147.449    2772.135
BsmtUnfSF             -1566.9667    531.862     -2.946      0.003   -2609.862    -524.072
TotalBsmtSF            8180.1354    827.618      9.884      0.000    6557.311    9802.960
HeatingQC              1337.9405    616.424      2.170      0.030     129.232    2546.649
CentralAir            -1369.8528    601.986     -2.276      0.023   -2550.251    -189.455
x1stFlrSF              7704.7889    944.780      8.155      0.000    5852.227    9557.350
x2ndFlrSF              1.246e+04   1050.930     11.861      0.000    1.04e+04    1.45e+04
LowQualFinSF            202.3725    508.206      0.398      0.691    -794.137    1198.882
GrLivArea              1.655e+04    810.470     20.426      0.000     1.5e+04    1.81e+04
BsmtFullBath            933.4592    683.820      1.365      0.172    -407.401    2274.320
BsmtHalfBath           -189.3039    488.487     -0.388      0.698   -1147.147     768.540
FullBath               1127.2087    798.953      1.411      0.158    -439.408    2693.826
HalfBath               

In [105]:
from sklearn.metrics import mean_squared_error

In [91]:
estimator = LinearRegression()

n_features = [1, 5, 10, 20, 30, 50, 60]
cv = [3, 5, 10]
data = []

for n in n_features:
    for c in cv:
        rfe = RFECV(estimator, cv=c, min_features_to_select=n)
        rfe = rfe.fit(X_train, y_train)
        y_= rfe.predict(X_test)
        mse = mean_squared_error(y_test, y_)
        score = rfe.score(X_test, y_test)
        
        row = {'cv':c, 'n_features': n, 'mse': mse, 'test_score' : score}
        data.append(row)

data = pd.DataFrame(data)


In [92]:
data

,cv,n_features,mse,test_score
0,3,1,4985375069.55,0.17
1,5,1,4991617386.64,0.17
2,10,1,5030397081.14,0.16
3,3,5,4985375069.55,0.17
4,5,5,4991617386.64,0.17
5,10,5,5030397081.14,0.16
6,3,10,4985375069.55,0.17
7,5,10,4991617386.64,0.17
8,10,10,1040031269450272598704519118848.00,-173526864550232686592.00
9,3,20,16134508554824526386211848192.00,-2692006252905718784.00


In [99]:
import time 

data.to_csv('linear_regression_' + time.strftime("%d_%m_%Y") +'.csv' )